In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import numpy as np
from scipy import interpolate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from google.colab import drive
import pickle
from scipy.io import wavfile
from scipy.signal import decimate
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
### Aggregation functions

def scaled_weights(model, scale):
    weights = model.get_weights()
    for layer_idx in range(len(weights)):
        weights[layer_idx] = weights[layer_idx] * scale
    return weights

def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

def choose_model(i):
  if 0 <= i <= 2:
    f = Uni_Modal(i)
  elif i == 3:
    f = Multi_Modal()
  elif i == 10:
    f = PS2W()
  elif i == 11:
    f = P2W()
  elif i == 12:
    f = S2W()
  return f


def choose_data(i):
  if i == 0:
    train_x = local_paired_phone_train
    train_y = local_paired_label_train
    test_x = global_paired_phone_test
    test_y = global_paired_label_test
  elif i == 1:
    train_x = local_paired_watch_train
    train_y = local_paired_label_train
    test_x = global_paired_watch_test
    test_y = global_paired_label_test
  elif i == 2:
    train_x = local_paired_audio_train
    train_y = local_paired_label_train
    test_x = global_paired_audio_test
    test_y = global_paired_label_test
  elif i == 3:
    train_x = [local_paired_watch_train,local_paired_phone_train,local_paired_audio_train]
    train_y = local_paired_label_train
    test_x = [global_paired_watch_test,global_paired_phone_test,global_paired_audio_test]
    test_y = global_paired_label_test
  elif i == 4:
    train_x = central_paired_phone_train
    train_y = central_paired_label_train
    test_x = global_paired_phone_test
    test_y = global_paired_label_test
  elif i == 5:
      train_x = central_paired_watch_train
      train_y = central_paired_label_train
      test_x = global_paired_watch_test
      test_y = global_paired_label_test
  elif i == 6:
      train_x = central_paired_audio_train
      train_y = central_paired_label_train
      test_x = global_paired_audio_test
      test_y = global_paired_label_test
  elif i == 7:
      train_x = [central_paired_watch_train, central_paired_phone_train,\
                 central_paired_audio_train]
      train_y = central_paired_label_train
      test_x = [global_paired_watch_test, global_paired_phone_test,\
                 global_paired_audio_test]
      test_y = global_paired_label_test
  return train_x,train_y, test_x, test_y

In [ ]:
def FedL(mode, Train_x, Train_y, Test_x, Test_y, comm_rounds = 100,local_epochs = 3,batch_size = 32, num_select_per_round = 20,num_clients = 20):
# mode => 0: Phone, 1:Watch, 2: Speaker, 3: Multi-Modal
  global_model = choose_model(mode)
  W_t_1 = global_model.get_weights()
  W_t = W_t_1

  num_samples =np.array([len(Train_y[idx]) for idx in range(num_clients)])

  global_acc = list()
  acc_out_list = list()

  for t in range(comm_rounds):
    scaled_weight_list = []
    #Randomly selecting "num_candidates" candidates from "num_clients" clients
    participant_idx = random.sample(range(num_clients), num_select_per_round)
    participant_idx = np.sort(participant_idx)
    W_t = global_model.get_weights()
    FedAvg_scales = num_samples/sum(num_samples[participant_idx])
    #loop through each client and create new local model
    for client in participant_idx:
      #local_model = global_model
      local_model = choose_model(mode)

      #set local model weight to the weight of the global model
      local_model.set_weights(W_t)
      #fit local model with client's data
      if mode == 3:
        local_model.fit([Train_x[0][client].reshape((-1,25,24,1)),Train_x[1][client],Train_x[2][client]],Train_y[client], epochs=local_epochs, verbose=0,batch_size = batch_size)
      elif mode == 10:
        local_model.fit([Train_x[0][client],Train_x[1][client]],Train_y[client], epochs=local_epochs, verbose=0,batch_size = batch_size)
      else:
        local_model.fit(Train_x[client],Train_y[client], epochs=local_epochs, verbose=0,batch_size = batch_size)

      # aggregation
      scaled_weight_list.append(scaled_weights(local_model,FedAvg_scales[client]))
      #K.clear_session()

    average_weights = sum_scaled_weights(scaled_weight_list)
    global_model.set_weights(average_weights)
    _,acc_FedAvg = global_model.evaluate(Test_x, Test_y,verbose = 0)
    #acc_FedAvg = global_model.evaluate(Test_x, Test_y,verbose = 0)
    global_acc.append(acc_FedAvg)
    print('communication round',t, '|', 'global accuracy:',acc_FedAvg)
    print('------------------------------------------------')

  return global_model, global_acc

In [ ]:
## Distributed training for Watch Modality

In [ ]:
mode = 1
train_x = local_train_x_w
train_y = local_train_y

test_x = central_test_x_w
test_y = central_test_y
global_model, global_acc = FedL(comm_rounds=100,mode= mode, Train_x = train_x, Train_y = train_y, Test_x = test_x, Test_y = test_y)

In [ ]:
## Distributed training for Phone Modality

In [ ]:
mode = 0
train_x = local_train_x_p
train_y = local_train_y

test_x = central_test_x_p
test_y = central_test_y
global_model, global_acc = FedL(comm_rounds=100,mode= mode, Train_x = train_x, Train_y = train_y, Test_x = test_x, Test_y = test_y)

In [ ]:
## Distributed training for Speaker Modality

In [ ]:
mode = 2
train_x = local_train_x_s
train_y = local_train_y

test_x = central_test_x_s
test_y = central_test_y
global_model, global_acc = FedL(comm_rounds=100,mode= mode, Train_x = train_x, Train_y = train_y, Test_x = test_x, Test_y = test_y)